In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from pathlib import Path
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <08E12B12-6183-307E-BDA0-374FA8EBA2C9> /Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torchvision/image.so
  Expected in:     <709C1DF5-D253-3C66-87E2-C99FD3A259DF> /Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/transformers/ut

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import resnet50
from torchvision.ops.feature_pyramid_network import FeaturePyramidNetwork, LastLevelMaxPool

class FPN(nn.Module):
    def __init__(self, num_classes=1, pretrained_backbone=True, backbone_weights_path=None):
        super(FPN, self).__init__()
        # 使用ResNet-50作为FPN的骨干网络
        backbone = resnet50(pretrained=False)  # 设置为不自动下载预训练模型
        if pretrained_backbone and backbone_weights_path:
            backbone.load_state_dict(torch.load(backbone_weights_path))  # 加载指定路径的权重文件

        # 保留 ResNet-50 的初始层（包含卷积、BN、ReLU 和 max pooling）
        self.initial_layers = nn.Sequential(
            backbone.conv1,
            backbone.bn1,
            backbone.relu,
            backbone.maxpool
        )

        # 定义 FPN 需要的不同分辨率特征图
        self.backbone_layers = nn.ModuleDict({
            'layer1': backbone.layer1,  # 输出通道数为256
            'layer2': backbone.layer2,  # 输出通道数为512
            'layer3': backbone.layer3,  # 输出通道数为1024
            'layer4': backbone.layer4   # 输出通道数为2048
        })
        
        # FPN 定义，in_channels_list 对应各层的输出通道数
        self.fpn = FeaturePyramidNetwork(
            in_channels_list=[256, 512, 1024, 2048],  # 对应每一层的输出通道数
            out_channels=256,
            extra_blocks=LastLevelMaxPool()
        )
        
        # 最终分类头
        self.classifier = nn.Conv2d(256, num_classes, kernel_size=1)

    def forward(self, x):
        # 通过初始层处理输入图像
        x = self.initial_layers(x)
        
        # 提取不同层级的特征
        features = {}
        for name, layer in self.backbone_layers.items():
            x = layer(x)
            features[name] = x

        # 将特征输入FPN
        fpn_outs = self.fpn(features)

        # 获取 P3 层输出用于最终的二分类分割
        p3_out = fpn_outs['layer1']  # 使用 layer1 输出作为主分辨率
        output = self.classifier(p3_out)
        
        # 上采样到原始图像尺寸
        output = F.interpolate(output, scale_factor=4, mode="bilinear", align_corners=False)
        return output

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from pathlib import Path
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score

# 自定义数据集类
class ContrailDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        super().__init__()
        self.image_dir = Path(image_dir)
        self.mask_dir = Path(mask_dir)
        self.files = [f for f in self.image_dir.iterdir() if f.is_file() and f.suffix in ['.png', '.jpg', '.jpeg']]
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        image_path = self.files[idx]
        mask_path = self.mask_dir / image_path.name

        image = Image.open(image_path).convert('RGB')  # 读取为RGB图像
        mask = Image.open(mask_path).convert('L')  # 读取为灰度图像

        if self.transform:
            image = self.transform(image)
            mask = transforms.ToTensor()(mask)

        mask = (mask > 0).float()  # 将掩码二值化
        return image, mask.unsqueeze(0)  # 在channel维度增加一个维度

# 定义Dice损失函数
class DiceLoss(nn.Module):
    def forward(self, inputs, targets, smooth=1):
        inputs = torch.sigmoid(inputs)
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        intersection = (inputs * targets).sum()
        dice = (2. * intersection + smooth) / (inputs.sum() + targets.sum() + smooth)
        return 1 - dice

# 计算 Precision, Recall, F1 Score, IoU
def calculate_metrics(preds, labels):
    preds = (torch.sigmoid(preds) > 0.5).cpu().numpy().flatten()  # 二值化预测
    labels = labels.cpu().numpy().flatten()

    precision = precision_score(labels, preds, zero_division=1)
    recall = recall_score(labels, preds, zero_division=1)
    f1 = f1_score(labels, preds, zero_division=1)
    iou = jaccard_score(labels, preds, zero_division=1)

    return precision, recall, f1, iou

# 计算平均 IoU（mIoU），对于二分类问题，mIoU 就是 IoU 本身
def calculate_mIoU(preds, labels):
    preds = (torch.sigmoid(preds) > 0.5).cpu().numpy()
    labels = labels.cpu().numpy()

    ious = []
    for i in range(preds.shape[0]):  # 针对每一张图片
        iou = jaccard_score(labels[i].flatten(), preds[i].flatten(), zero_division=1)
        ious.append(iou)
    
    return sum(ious) / len(ious)

def calculate_accuracy(preds, labels):
    # 使用Sigmoid并将输出二值化
    preds = (torch.sigmoid(preds) > 0.5).float()  # 二值化预测
    # 确保预测和标签维度一致
    preds = preds.view(-1)
    labels = labels.view(-1)

    # 计算正确的预测像素总数
    correct = (preds == labels).sum().item()
    # 计算总像素数
    total = labels.size(0)

    # 返回准确率
    acc = correct / total
    return acc

# 训练函数
def train_model(model, train_loader, valid_loader, loss_fn, optimizer, num_epochs, device):
    best_f1 = 0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        train_acc = 0

        # Training phase
        for images, masks in train_loader:
            images, masks = images.to(device), masks.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_fn(outputs, masks)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            acc = calculate_accuracy(outputs, masks)
            train_acc += acc

        # Validation phase
        model.eval()
        valid_loss = 0
        valid_acc = 0
        valid_precision = 0
        valid_recall = 0
        valid_f1 = 0
        valid_iou = 0
        valid_miou = 0

        with torch.no_grad():
            for images, masks in valid_loader:
                images, masks = images.to(device), masks.to(device)
                outputs = model(images)

                loss = loss_fn(outputs, masks)
                valid_loss += loss.item()

                acc = calculate_accuracy(outputs, masks)
                valid_acc += acc

                precision, recall, f1, iou = calculate_metrics(outputs, masks)
                valid_precision += precision
                valid_recall += recall
                valid_f1 += f1
                valid_iou += iou
                valid_miou += calculate_mIoU(outputs, masks)

        # 平均值
        avg_train_loss = train_loss / len(train_loader)
        avg_train_acc = train_acc / len(train_loader)
        avg_valid_loss = valid_loss / len(valid_loader)
        avg_valid_acc = valid_acc / len(valid_loader)
        avg_valid_precision = valid_precision / len(valid_loader)
        avg_valid_recall = valid_recall / len(valid_loader)
        avg_valid_f1 = valid_f1 / len(valid_loader)
        avg_valid_iou = valid_iou / len(valid_loader)
        avg_valid_miou = valid_miou / len(valid_loader)

        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Train Acc: {avg_train_acc:.4f}, '
              f'Val Loss: {avg_valid_loss:.4f}, Val Acc: {avg_valid_acc:.4f}, '
              f'Precision: {avg_valid_precision:.4f}, Recall: {avg_valid_recall:.4f}, '
              f'F1 Score: {avg_valid_f1:.4f}, IoU: {avg_valid_iou:.4f}, mIoU: {avg_valid_miou:.4f}')

        # 保存最好的模型
        if avg_valid_f1 > best_f1:
            best_f1 = avg_valid_f1
            torch.save(model.state_dict(), 'FPN_model.pth')

# 设置参数
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_dir = '/Users/camus/Desktop/trainX/image'
mask_dir = '/Users/camus/Desktop/trainX/maskimage'

batch_size = 4
num_epochs = 100

# 图像预处理，直接使用256x256的尺寸
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # 将图像调整为256x256
    transforms.ToTensor(),
])

# 加载数据集
dataset = ContrailDataset(image_dir, mask_dir, transform=transform)
train_size = int(0.75 * len(dataset))
valid_size = len(dataset) - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size)

# 指定权重文件的路径
backbone_weights_path = '/Volumes/Vettel/学习/预训练模型/resnet50-0676ba61.pth'

# 初始化FPN模型和损失函数
model = FPN(num_classes=1, pretrained_backbone=True, backbone_weights_path=backbone_weights_path).to(device)
loss_fn = DiceLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 训练模型
train_model(model, train_loader, valid_loader, loss_fn, optimizer, num_epochs, device)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch [1/100], Train Loss: 0.4009, Train Acc: 0.8821, Val Loss: 0.3057, Val Acc: 0.9340, Precision: 0.6100, Recall: 0.8273, F1 Score: 0.6943, IoU: 0.5383, mIoU: 0.5239
Epoch [2/100], Train Loss: 0.2692, Train Acc: 0.9443, Val Loss: 0.1950, Val Acc: 0.9623, Precision: 0.7882, Recall: 0.8313, F1 Score: 0.8050, IoU: 0.6796, mIoU: 0.6832
Epoch [3/100], Train Loss: 0.2695, Train Acc: 0.9454, Val Loss: 0.2602, Val Acc: 0.9576, Precision: 0.8787, Recall: 0.6470, F1 Score: 0.7398, IoU: 0.5922, mIoU: 0.5795
Epoch [4/100], Train Loss: 0.2334, Train Acc: 0.9516, Val Loss: 0.5091, Val Acc: 0.9320, Precision: 0.9563, Recall: 0.3444, F1 Score: 0.4909, IoU: 0.3363, mIoU: 0.3510
Epoch [5/100], Train Loss: 0.2430, Train Acc: 0.9491, Val Loss: 0.2235, Val Acc: 0.9508, Precision: 0.6925, Recall: 0.8959, F1 Score: 0.7765, IoU: 0.6416, mIoU: 0.6562
Epoch [6/100], Train Loss: 0.2167, Train Acc: 0.9552, Val Loss: 0.1836, Val Acc: 0.9655, Precision: 0.8137, Recall: 0.8263, F1 Score: 0.8165, IoU: 0.6959, mIoU:

Epoch [50/100], Train Loss: 0.2309, Train Acc: 0.9502, Val Loss: 0.2398, Val Acc: 0.9573, Precision: 0.7735, Recall: 0.7611, F1 Score: 0.7602, IoU: 0.6237, mIoU: 0.6194
Epoch [51/100], Train Loss: 0.1886, Train Acc: 0.9620, Val Loss: 0.3653, Val Acc: 0.9457, Precision: 0.9214, Recall: 0.5023, F1 Score: 0.6347, IoU: 0.4770, mIoU: 0.4746
Epoch [52/100], Train Loss: 0.1669, Train Acc: 0.9660, Val Loss: 0.1936, Val Acc: 0.9659, Precision: 0.8882, Recall: 0.7429, F1 Score: 0.8064, IoU: 0.6799, mIoU: 0.6683
Epoch [53/100], Train Loss: 0.1439, Train Acc: 0.9715, Val Loss: 0.2074, Val Acc: 0.9635, Precision: 0.8390, Recall: 0.7605, F1 Score: 0.7925, IoU: 0.6650, mIoU: 0.6430
Epoch [54/100], Train Loss: 0.1450, Train Acc: 0.9699, Val Loss: 0.3004, Val Acc: 0.9281, Precision: 0.6370, Recall: 0.8041, F1 Score: 0.6996, IoU: 0.5501, mIoU: 0.6045
Epoch [55/100], Train Loss: 0.1764, Train Acc: 0.9637, Val Loss: 0.2107, Val Acc: 0.9619, Precision: 0.8565, Recall: 0.7351, F1 Score: 0.7892, IoU: 0.6571,

Epoch [99/100], Train Loss: 0.0587, Train Acc: 0.9883, Val Loss: 0.1653, Val Acc: 0.9689, Precision: 0.8460, Recall: 0.8282, F1 Score: 0.8347, IoU: 0.7227, mIoU: 0.7194
Epoch [100/100], Train Loss: 0.0579, Train Acc: 0.9887, Val Loss: 0.1648, Val Acc: 0.9687, Precision: 0.8341, Recall: 0.8413, F1 Score: 0.8352, IoU: 0.7239, mIoU: 0.7185
